In [1]:
from sklearn.linear_model import SGDRegressor
from functools import lru_cache
from pymorphy2 import MorphAnalyzer
import pandas as pd
import numpy as np
import re
from tqdm import tqdm
import random
from sklearn.feature_extraction.text import TfidfVectorizer
from scipy.sparse import hstack

random.seed(42)
np.random.seed(42)

In [2]:
df_train = pd.read_csv('vacancies_train.csv.zip')
df_test = pd.read_csv('vacancies_test.csv.zip', index_col=0)

In [3]:
# # Препроцессинг
m = MorphAnalyzer()

def words_only(text):
    res = re.sub(r'[^\/0-9а-яёa-z ]+', '', text.lower()).strip()
    return re.sub(r' +', ' ', res).split()

@lru_cache(maxsize=128)
def lemmatize_word(token, pymorphy=m):
    return pymorphy.parse(token)[0].normal_form

def lemmatize_text(text):
    return ' '.join([lemmatize_word(w) for w in words_only(text)])

def add_lemmas_column(df):
    df_res = df.copy()
    with tqdm(total=df_res.shape[0]) as pbar:
        df_res['description_lemmas'] = df_res.description.apply(
            lambda x: (lemmatize_text(x), pbar.update(1))[0]
        )
    return df_res

In [4]:
X_train = add_lemmas_column(df_train).drop('salary', axis=1)
y_train = df_train.salary
X_test = add_lemmas_column(df_test)

100%|██████████████████████████████████████████████████████████████████████████████| 5543/5543 [01:23<00:00, 66.49it/s]


In [5]:
X_train

,name,experience,schedule,employment,description,key_skills,description_lemmas
0,Водитель-экспедитор,От 3 до 6 лет,Полный день,Полная занятость,"Водитель-экспедитор категории В,С (а\м ГАЗЕЛЬ ...",[],водительэкспедитор категория вс ама газель тен...
1,Кладовщик-комплектовщик/оператор 1С,От 1 года до 3 лет,Полный день,Полная занятость,Обязанности: Списывание материалов в произво...,"['знание пк', 'работа в 1С', 'Работа в команде...",обязанность списывание материал в производство...
2,"Продавец магазина ""Магнит"" (Кировский р-он, В....",Нет опыта,Сменный график,Полная занятость,"В крупнейшую Федеральную сеть магазинов ""Магни...",[],в крупный федеральный сеть магазин магнит треб...
3,Администратор магазина (ТЦ Spar),От 1 года до 3 лет,Полный день,Полная занятость,Крупная федеральная сеть магазинов мужской и ж...,[],крупный федеральный сеть магазин мужской и жен...
4,Мойщик посуды (кулинария),Нет опыта,Сменный график,Полная занятость,Обязанности: Уборка производственных площадей...,"['Пользователь ПК', 'Грамотная речь', 'Работа ...",обязанность уборка производственный площадь на...
...,...,...,...,...,...,...,...
16624,Полицейский специального полка полиции,Нет опыта,Полный день,Полная занятость,Обязанности: обеспечение общественного поряд...,"['Грамотная речь', 'Обучение и развитие', 'Упр...",обязанность обеспечение общественный порядок и...
16625,Врач стоматолог-ортопед (г. Новосибирск),Нет опыта,Полный день,Полная занятость,Обязанности: Оказывает квалифицированную меди...,['Медицинская документация'],обязанность оказывать квалифицировать медицинс...
16626,Фармацевт,От 1 года до 3 лет,Полный день,Полная занятость,Условия: официальное оформление по ТК РФ; гра...,"['Пользователь ПК', 'Работа в команде', 'Грамо...",условие официальный оформление по тк рф график...
16627,Геодезист (автодороги),От 1 года до 3 лет,Вахтовый метод,Полная занятость,Обязанности: своевременное и качественное вып...,[],обязанность своевременный и качественный выпол...


In [6]:
# # Каждый из значащих столбцов данных векторизуем TF-IDF
tfidf_words = TfidfVectorizer(analyzer='word', ngram_range=(1, 2), 
                              lowercase=True, sublinear_tf=True)
tfidf_chars = TfidfVectorizer(analyzer='char', ngram_range=(1, 5), 
                              lowercase=True, sublinear_tf=True)
tfidf_names = TfidfVectorizer(analyzer='char', ngram_range=(1, 5), 
                              lowercase=True, sublinear_tf=True)

In [7]:
%%time
a = tfidf_words.fit_transform(X_train.description_lemmas)
b = tfidf_chars.fit_transform(X_train.description)
c = tfidf_names.fit_transform(X_train.name)
X_train_s = hstack([a, b, c])

Wall time: 1min 8s


In [13]:
X_train_s

<16629x1536994 sparse matrix of type '<class 'numpy.float64'>'
	with 51235447 stored elements in Compressed Sparse Row format>

In [11]:
# # Обучаем стохастическим градиентом
%%time
reg = SGDRegressor(max_iter=2000)
reg.fit(X_train_s, y_train)

Wall time: 6min 53s


C:\Users\dimam\anaconda3\lib\site-packages\sklearn\linear_model\_stochastic_gradient.py:1503: ConvergenceWarning: Maximum number of iteration reached before convergence. Consider increasing max_iter to improve the fit.
  warnings.warn(


SGDRegressor(max_iter=2000)

In [24]:
def pred_to_csv(pred, name):
    a = pd.Series(pred, name="salary")
    a.index.name = "id"
    a.to_csv(f"C:/Users/dimam/{name}.csv")

In [12]:
%%time
a2 = tfidf_words.transform(X_test.description_lemmas)
b2 = tfidf_chars.transform(X_test.description)
c2 = tfidf_names.transform(X_test.name)
X_test_s = hstack([a2, b2, c2])

Wall time: 19.9 s


In [26]:
# # Переносим предсказанную ЗП в файл с ответом
pred = reg.predict(X_test_s)
pred_to_csv(pred, "res")

In [ ]:
# # Финальный скор - около 0.45 (Максимальный скор в соревновании - 0.55)